<a href="https://colab.research.google.com/github/akhaire21/Semi-Supervised-Ensemble-Model/blob/main/Microglia_AD_Research_DeepGWAS_DataPrep_FinalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive = drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Start R session
%reload_ext rpy2.ipython

In [3]:
%%R
# Install required R packages
install.packages('R.utils')
install.packages('sqldf')

(as ‘lib’ is unspecified)







	‘/tmp/RtmprnsUqA/downloaded_packages’

(as ‘lib’ is unspecified)







	‘/tmp/RtmprnsUqA/downloaded_packages’



# Load AnnotatedFeaturesBellenguez_21.csv

In [4]:
%%R
library(data.table)
AnnotatedFeaturesBellenguez <- fread("/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_21.csv",showProgress = FALSE) 
nrow(AnnotatedFeaturesBellenguez) #21061706

[1] 21061706


In [5]:
%%R
head(AnnotatedFeaturesBellenguez)

   chromosome  start    end     NA sweeps C1 C2 C3 C4 C5 C6 C7 cCRE
1:       chr1 529825 529826 0.9626      0  0  0  0  0  0  0  0    0
2:       chr1 531142 531143 0.2670      0  0  0  0  0  0  0  0    0
3:       chr1 566327 566328 0.8214      0  0  0  0  0  0  0  0    0
4:       chr1 581537 581538 0.3411      0  0  0  0  0  0  0  0    0
5:       chr1 661906 661907 0.3542      0  0  0  0  0  0  0  0    0
6:       chr1 662613 662614 0.6424      0  0  0  0  0  0  0  0    0
   chiapet.loops footprints dhs.peaks rad21 tfbinding FB1 fire40kb_Adult
1:             0          0         0     0         0   0              0
2:             0          0         0     0         0   0              0
3:             0          0         0     0         0   0              0
4:             0          0         0     0         0   0              0
5:             0          0         0     0         0   0              0
6:             0          0         0     0         0   0              0
   fire40kb_F

In [6]:
##Remove duplicates based on column "chromosome","start"
%%R
AnnotatedFeaturesBellenguez <- AnnotatedFeaturesBellenguez[!duplicated(AnnotatedFeaturesBellenguez[ , c("chromosome","start")]),]
nrow(AnnotatedFeaturesBellenguez) #21,016,274  (# Duplicates = 21,061,706 - 21,016,274 = 45,432)

[1] 21016274


# Load Bellenguez_21_BedLifted_IdAllelesIncluded.bed
This file is used to get RSIDs which are missing in Bellenguez.csv file

In [7]:
%%R
library(data.table)
BEDFile <- fread("/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/Bellenguez_21_BedLifted_IdAllelesIncluded.bed", showProgress = FALSE) 
nrow(BEDFile) ##21,061,706

[1] 21061706


In [8]:
%%R
head(BEDFile)

     V1     V2     V3     V4           V5  V6 V7
1: chr1 529825 529826 0.9626  rs888953847   T  C
2: chr1 531142 531143 0.2670 rs1040232850 CTG  C
3: chr1 566327 566328 0.8214 rs1390538076   A  G
4: chr1 581537 581538 0.3411 rs1250812823   A  G
5: chr1 661906 661907 0.3542  rs991450070   C  G
6: chr1 662613 662614 0.6424  rs151190501   A  G


In [9]:
%%R
colnames(BEDFile) <- c("chr", "start", "end", "p_value", "RSID",	"effect_allele",	"other_allele")
head(BEDFile)

    chr  start    end p_value         RSID effect_allele other_allele
1: chr1 529825 529826  0.9626  rs888953847             T            C
2: chr1 531142 531143  0.2670 rs1040232850           CTG            C
3: chr1 566327 566328  0.8214 rs1390538076             A            G
4: chr1 581537 581538  0.3411 rs1250812823             A            G
5: chr1 661906 661907  0.3542  rs991450070             C            G
6: chr1 662613 662614  0.6424  rs151190501             A            G


In [10]:
##Remove duplicates based on column V1 & V2
%%R
BEDFile <- BEDFile[!duplicated(BEDFile[ , c("chr","start")]),]
nrow(BEDFile) #21,016,274  (# Duplicates = 21,061,706 - 21,016,274 = 45,432)

[1] 21016274


In [11]:
%%R
BEDFile <- BEDFile[, c("chr", "start", "RSID")]
head(BEDFile)

    chr  start         RSID
1: chr1 529825  rs888953847
2: chr1 531142 rs1040232850
3: chr1 566327 rs1390538076
4: chr1 581537 rs1250812823
5: chr1 661906  rs991450070
6: chr1 662613  rs151190501


In [12]:
%%R
AnnotatedFeaturesBellenguez <- merge(AnnotatedFeaturesBellenguez, BEDFile, by.x=c('chromosome','start'), by.y=c('chr','start'))
nrow(AnnotatedFeaturesBellenguez)

[1] 21016274


In [13]:
%%R
names(AnnotatedFeaturesBellenguez)[names(AnnotatedFeaturesBellenguez) == "NA"] <- "pvalue"
head(AnnotatedFeaturesBellenguez)

   chromosome  start    end pvalue sweeps C1 C2 C3 C4 C5 C6 C7 cCRE
1:       chr1 229820 229821 0.2303      0  0  0  0  0  0  0  0    0
2:       chr1 529825 529826 0.9626      0  0  0  0  0  0  0  0    0
3:       chr1 531142 531143 0.2670      0  0  0  0  0  0  0  0    0
4:       chr1 566327 566328 0.8214      0  0  0  0  0  0  0  0    0
5:       chr1 581537 581538 0.3411      0  0  0  0  0  0  0  0    0
6:       chr1 661906 661907 0.3542      0  0  0  0  0  0  0  0    0
   chiapet.loops footprints dhs.peaks rad21 tfbinding FB1 fire40kb_Adult
1:             0          0         0     0         0   0              0
2:             0          0         0     0         0   0              0
3:             0          0         0     0         0   0              0
4:             0          0         0     0         0   0              0
5:             0          0         0     0         0   0              0
6:             0          0         0     0         0   0              0
   fire40kb_F

In [14]:
%%R
cols <- colnames(AnnotatedFeaturesBellenguez)
cols <- append(cols,'Key')
len=length(cols)
df <- data.frame(matrix(ncol = len, nrow = 0))
colnames(df) <- cols
print(df)

 [1] chromosome          start               end                
 [4] pvalue              sweeps              C1                 
 [7] C2                  C3                  C4                 
[10] C5                  C6                  C7                 
[13] cCRE                chiapet.loops       footprints         
[16] dhs.peaks           rad21               tfbinding          
[19] FB1                 fire40kb_Adult      fire40kb_Fetal     
[22] MDT_neuron          open_adult          open_fetal         
[25] Organoid_11         PEC_enhancers       phylop             
[28] superfire40kb_Adult superfire40kb_Fetal eQTL               
[31] RSID                Key                
<0 rows> (or 0-length row.names)


In [15]:
%%R
i <- 1

while (i <= 22){
fnamein = gsub(' ','',paste("/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr", i, ".out.BED_CADD_FATHMM.gz"))
caddft <- fread(fnamein,select = c(3,4,7,11,12), showProgress = FALSE)
print(fnamein)
colnames(caddft) <- c("chr", "start", "RSID", "CADD", "FATHMM")
print(nrow(caddft))

chr <- merge(AnnotatedFeaturesBellenguez, caddft, by.x=c('RSID'), by.y=c('RSID'))
chr$Key <- gsub(' ','',paste(chr$chromosome,':',chr$start))
print(nrow(chr))

df <- rbind(df,chr)
print(nrow(df))

i <- i+1
}

fnameout = "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM.gz"
print(fnameout)
write.table(df, fnameout, quote = F, sep="\t", col.names = NA, row.names = T)
print(head(df))

[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr1.out.BED_CADD_FATHMM.gz"
[1] 1561192
[1] 1561192
[1] 1561192
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr2.out.BED_CADD_FATHMM.gz"
[1] 1690591
[1] 1690591
[1] 3251783
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr3.out.BED_CADD_FATHMM.gz"
[1] 1435976
[1] 1435976
[1] 4687759
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr4.out.BED_CADD_FATHMM.gz"
[1] 1418539
[1] 1418539
[1] 6106298
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr5.out.BED_CADD_FATHMM.gz"
[1] 1292120
[1] 1292120
[1] 7398418
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmScores/chr6.out.BED_CADD_FATHMM.gz"
[1] 1267240
[1] 1267240
[1] 8665658
[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/MatchedCaddFathmmS

In [16]:
%%R
cols <- colnames(df)
cols

 [1] "RSID"                "chromosome"          "start.x"            
 [4] "end"                 "pvalue"              "sweeps"             
 [7] "C1"                  "C2"                  "C3"                 
[10] "C4"                  "C5"                  "C6"                 
[13] "C7"                  "cCRE"                "chiapet.loops"      
[16] "footprints"          "dhs.peaks"           "rad21"              
[19] "tfbinding"           "FB1"                 "fire40kb_Adult"     
[22] "fire40kb_Fetal"      "MDT_neuron"          "open_adult"         
[25] "open_fetal"          "Organoid_11"         "PEC_enhancers"      
[28] "phylop"              "superfire40kb_Adult" "superfire40kb_Fetal"
[31] "eQTL"                "chr"                 "start.y"            
[34] "CADD"                "FATHMM"              "Key"                


In [17]:
%%R
df <- df[,-33] 
df <- df[,-32] 
names(df)[names(df) == "start.x"] <- "start"
cols <- colnames(df)
cols

 [1] "RSID"                "chromosome"          "start"              
 [4] "end"                 "pvalue"              "sweeps"             
 [7] "C1"                  "C2"                  "C3"                 
[10] "C4"                  "C5"                  "C6"                 
[13] "C7"                  "cCRE"                "chiapet.loops"      
[16] "footprints"          "dhs.peaks"           "rad21"              
[19] "tfbinding"           "FB1"                 "fire40kb_Adult"     
[22] "fire40kb_Fetal"      "MDT_neuron"          "open_adult"         
[25] "open_fetal"          "Organoid_11"         "PEC_enhancers"      
[28] "phylop"              "superfire40kb_Adult" "superfire40kb_Fetal"
[31] "eQTL"                "CADD"                "FATHMM"             
[34] "Key"                


In [18]:
%%R
df$Key <- gsub(' ','',paste(df$chromosome,':',df$start))
head(df)

           RSID chromosome     start       end  pvalue sweeps C1 C2 C3 C4 C5 C6
1: rs1000000326       chr1 189920805 189920806 0.08383      0  0  0  0  0  0  0
2: rs1000003820       chr1  45061723  45061724 0.67710      0  0  0  0  0  0  0
3: rs1000006697       chr1 155902677 155902678 0.11860      0  0  0  0  0  0  0
4: rs1000011051       chr1  13723909  13723910 0.15460      0  0  0  0  0  0  0
5: rs1000011868       chr1  56070056  56070057 0.20130      0  0  0  0  0  0  0
6: rs1000017931       chr1 179939123 179939124 0.65290      0  0  0  0  0  0  0
   C7 cCRE chiapet.loops footprints dhs.peaks rad21 tfbinding FB1
1:  0    0             0          0         0     0         0   0
2:  0    0             0          0         0     0         0   0
3:  0    0             0          0         0     0         0   0
4:  0    0             0          0         0     0         0   0
5:  0    0             0          0         0     0         0   0
6:  0    0             0          0         

In [19]:
%%R
fnameout = "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM.gz"
print(fnameout)
write.table(df, fnameout, quote = F, sep="\t", col.names = NA, row.names = T)
print(head(df))

[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM.gz"
           RSID chromosome     start       end  pvalue sweeps C1 C2 C3 C4 C5 C6
1: rs1000000326       chr1 189920805 189920806 0.08383      0  0  0  0  0  0  0
2: rs1000003820       chr1  45061723  45061724 0.67710      0  0  0  0  0  0  0
3: rs1000006697       chr1 155902677 155902678 0.11860      0  0  0  0  0  0  0
4: rs1000011051       chr1  13723909  13723910 0.15460      0  0  0  0  0  0  0
5: rs1000011868       chr1  56070056  56070057 0.20130      0  0  0  0  0  0  0
6: rs1000017931       chr1 179939123 179939124 0.65290      0  0  0  0  0  0  0
   C7 cCRE chiapet.loops footprints dhs.peaks rad21 tfbinding FB1
1:  0    0             0          0         0     0         0   0
2:  0    0             0          0         0     0         0   0
3:  0    0             0          0         0     0         0   0
4:  0    0             0          0         0     0         0   0
5:

In [20]:
%%R
s <- subset(AnnotatedFeaturesBellenguez, !(RSID %in% df$RSID))
nrow(s) #1,352,544

[1] 1352544


In [21]:
%%R
rm(AnnotatedFeaturesBellenguez)

In [22]:
%%R
l = list(df,s)
df2 <- rbindlist(l, use.names=TRUE, fill=TRUE)
#df2 <- df2[,-1] 
nrow(df2)

[1] 21016274


In [23]:
%%R
fnameout = "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM_FULL.gz"
print(fnameout)
write.table(df2, fnameout, quote = F, sep="\t", col.names = NA, row.names = T)

[1] "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM_FULL.gz"


In [24]:
%%R
rm(df)
rm(s)
rm(AnnotatedFeaturesBellenguez)

In [25]:
%%R
#library(data.table)
#fname = "/content/drive/MyDrive/Regeneron_STS_2023/Model/Input_Data/AnnotatedFeaturesBellenguez_CADD_FATHMM_FULL.gz"
#df2 <- fread(fname, showProgress = FALSE)
nrow(df2) #21016274

[1] 21016274


In [ ]:
%%R
df2 <- df2[,-1] 
head(df2)

In [ ]:
%%R
dfld <- data.frame(matrix(ncol = 2, nrow = 0))
x <- c("Key", "ld_sum_final")
colnames(dfld) <- x

LJoin with chrNN.out.BED_CADD_FATHMM.gz to get CADD and FATHMM scores
Join with chrNN_final.csv AND chrNN_ld_known.csv to get ld_sum_final AND ld_sum_known